# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [2]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [3]:
iterations = 5
ori_data = np.load('../OUTPUT/PSM_T1_finetuning/samples/PSM_T1_finetuning_ground_truth_24_train.npy')
print("ori_data_shape:",ori_data.shape)
# ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')
fake_data = np.load('../OUTPUT/PSM_T1_finetuning/ddpm_fake_PSM_T1_finetuning.npy')
print("fale_data_shape:",fake_data.shape)

ori_data_shape: (457, 24, 25)
fale_data_shape: (500, 24, 25)


## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [4]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:fake_data.shape[0]], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 7.810467442832926 

Iter 1:  context-fid = 7.795126499436941 

Iter 2:  context-fid = 5.76573576821573 

Iter 3:  context-fid = 6.748501374940715 

Iter 4:  context-fid = 7.983092493865113 

Final Score:  7.220584715858285 ± 1.1777701973097587


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [5]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [6]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 20.05929245075515 

Iter 1:  cross-correlation = 20.869017018057058 

Iter 2:  cross-correlation = 22.51259849486694 

Iter 3:  cross-correlation = 19.930814817493665 

Iter 4:  cross-correlation = 21.425391889572644 

Final Score:  20.959422934149092 ± 1.317850156381414
